# INTRODUCTION

*Cliente*: Organización independiente pero adscrita al Ministerio de Trabajo.

*Objetivo*: Analizar y visualizar datos relacionados con el paro nacional en España.

*Variables*: 
1. Pandemia
2. Género
3. Nivel de formación

*Catálogo de obtenicón de datos*: "Mercado Laboral y Salarios" 

*Orden de ETL*:
1. Data Lake
2. Data Warehouse
3. Web Scraping

*Visualizaciones*: 3 Dashboards en PowerBI 
1. Información General del Paro
2. Efectos de la Pandemia en el empleo
3. Diferencia de Paro según el nivel de formación alcanzado

# Data lake

In [ ]:
import pandas as pd

In [ ]:
DataLake = pd.read_csv("ETL-project-main\ETL_project-postal_codes.csv", sep=";")
DataLake.tail()

En este punto del proceso, no hemos realizado transformaciones ni limpieza significativa de los datos. El objetivo principal del Data Lake es capturar y conservar todos los datos en su formato original, de manera que posteriormente podamos realizar el proceso de Extracción, Transformación y Carga (ETL) en el Data Warehouse.

En la etapa del Data Lake, nos aseguramos de tener todos los datos disponibles para su posterior análisis, sin importar si provienen de diferentes fuentes o si están en formatos no estructurados. Una vez que los datos están almacenados en el Data Lake, podemos continuar con las siguientes etapas del proyecto, como la limpieza, transformación y carga en el Data Warehouse.

# Data Warehouse

In [ ]:
file = open("ETL-project-main/ETL_project-postal_codes.csv", encoding="latin1")
extension=len(file.readlines())
file.close()
lineas_de_texto=[]
file = open("ETL-project-main/ETL_project-postal_codes.csv", encoding="utf-8")
for i in range(extension):
    fila=[]
    variable = file.readline().split(";")
    for item in variable:
        if "/" in item:
            fila.extend(item.split("/"))
        elif item.strip().isdigit() == False:
            fila.extend([item,item])
        else:
            fila.append(item)
    lineas_de_texto.append(fila)
file.close()

En esta etapa del proyecto, nos encontramos trabajando en el Data Warehouse, donde llevamos a cabo el proceso de transformación de datos antes de cargarlos para su análisis y generación de informes.

El código proporcionado realiza una serie de transformaciones en el conjunto de datos contenido en el archivo "ETL_project-postal_codes.csv". La intención es preparar los datos para que sean más adecuados para el análisis posterior. A continuación, se describen las principales transformaciones que se están realizando:

1. Lectura del archivo: Primero, se lee el archivo "ETL_project-postal_codes.csv" utilizando la función `open()`, y se cuentan las líneas para determinar el tamaño del archivo.

2. Procesamiento de las líneas de texto: Luego, se procede a leer el archivo nuevamente, esta vez utilizando el conjunto de caracteres "utf-8" para asegurar una correcta interpretación de los caracteres especiales. Se recorren las líneas del archivo en un bucle `for`, y para cada línea se realizan las siguientes operaciones:

    a. Separación de elementos: Cada línea se divide utilizando el delimitador ";" para obtener una lista de elementos.

    b. Tratamiento de fechas: Si un elemento contiene una barra ("/"), se asume que representa una fecha en formato día/mes, por lo que se divide el elemento en dos partes (día y mes) y se agrega cada parte como elementos separados en la fila.

    c. Duplicación de elementos no numéricos: Si el elemento no es un número, se duplica en la fila. Esto podría deberse a que el dato es importante y se necesita más de una vez en el análisis posterior.

3. Almacenamiento de los datos transformados: Cada fila procesada se agrega a una lista llamada "lineas_de_texto", que contendrá todas las filas transformadas del archivo.

Es importante tener en cuenta que en este punto, aún no se ha llevado a cabo la carga de los datos transformados en el Data Warehouse. El objetivo de esta sección del código es realizar las transformaciones necesarias para que los datos estén limpios y estructurados adecuadamente antes de cargarlos en la siguiente etapa del proceso.

Una vez que se han realizado todas las transformaciones, el siguiente paso sería realizar la carga de los datos en el Data Warehouse, donde estarán listos para su análisis y generación de informes.

## Warehouse st.1 - Primera version CSV

In [ ]:
archivo=open('archivo_salida.csv',"w",encoding="utf-8")
archivo.write("Provincia_local,Provincia,Ciudad_local,Ciudad,CP\n")
for i in lineas_de_texto:
    archivo.write((",").join(i))

In [ ]:
provinces=pd.read_csv('archivo_salida.csv')
provinces.tail()

En esta etapa del Data Warehouse, estamos generando la primera versión de un archivo CSV que contiene datos transformados y limpios. El objetivo es preparar los datos para su posterior análisis y utilización.

El código proporcionado realiza las siguientes acciones:

1. Creación del archivo de salida: Se crea un nuevo archivo llamado 'archivo_salida.csv' utilizando la función `open()` con el modo de escritura ("w") y especificando la codificación "utf-8". Este archivo se utilizará para almacenar los datos transformados.

2. Escritura de la cabecera: Se escribe la primera línea en el archivo, que contiene los nombres de las columnas o campos. En este caso, los nombres son "Provincia_local", "Provincia", "Ciudad_local", "Ciudad" y "CP", que representan las diferentes partes de los datos transformados.

3. Escritura de los datos transformados: Luego, se recorre la lista "lineas_de_texto" que contiene los datos transformados generados en la etapa anterior. Para cada línea (fila) en la lista, se escriben los elementos (columnas) en el archivo 'archivo_salida.csv', separados por comas. Esto se logra mediante el método `join()` que une los elementos de la lista utilizando "," como separador.

4. Lectura de los datos transformados: Finalmente, se utiliza la biblioteca `pandas` para leer el archivo 'archivo_salida.csv' en un DataFrame llamado "provinces". Esto permitirá tener los datos listos para su análisis y manipulación en el entorno de trabajo de Python.

Es importante destacar que esta es la "Warehouse st.1", lo que significa que es la primera versión del archivo CSV con los datos transformados. En futuras etapas del Data Warehouse, es posible que se realicen más transformaciones o ajustes antes de que los datos estén completamente listos para su análisis y carga en sistemas de análisis como Power BI o herramientas similares.

Al usar un archivo CSV como formato de almacenamiento, los datos se vuelven más accesibles y fáciles de compartir con otros equipos o herramientas para su posterior análisis y visualización.


## QC del df

In [ ]:
def report(df):
    import numpy as np
    import pandas.api.types as api
    reemplazos = {" ": "", ',': '',"'": '',"-": "","\\x2E": "","\\x27": "","\\x28":"","\\x29": "", '\\x3F':""}
    Length=len(df)
    file=open("Load_report.txt", "w", encoding="utf-8")
    for i in range(len(df.columns)):
        x=df[df.columns[i]]
        name=df.columns[i]
        file.write(f"{name}\n{Length} filas\nValores no nulos:\t{x.count()}\nValores unicos:\t{x.nunique()}\nValores nulos:\t{x.isnull().sum()}\n")
        if api.is_numeric_dtype(df[df.columns[i]].dtype):
            Real=df[df.columns[i]].apply(np.isreal)==False
            file.write(f"Valores no reales:\t{Real.sum()}\n")
            file.write("\n")
        elif api.is_string_dtype(x.dtype):
            file.write(f"Valores no alfanumericos:\t{df[df.replace(reemplazos, regex=True)[df.columns[i]].str.isalnum() == False][df.columns[i]].value_counts()}\n")
            file.write("\n")
    file.close()
    print(f"Reporte de carga de datos generado en\n Files/Load_report.txt")

In [ ]:
report(provinces)   #txt generado para revision de valores anomalos

In [ ]:
provinces[provinces["Provincia_local"]=="﻿Araba"]  # muestra de fila con valor anomalo

In [ ]:
provinces.loc[0,"Provincia_local"] = "Araba"  # reescritura de valor en campo 'Provincia_local'

In [ ]:
provinces[provinces["Provincia_local"]=="﻿Araba"] # rechequeo de existencia de filas con valores anomalos

En esta etapa del proyecto, se está realizando un control de calidad (QC) del DataFrame `provinces` con el objetivo de identificar posibles valores anómalos o problemas en los datos. El código utiliza la biblioteca `pandas` y algunas funciones adicionales para generar un reporte de calidad y, si es necesario, corregir algunos valores anómalos encontrados.

El código del control de calidad realiza las siguientes acciones:

1. Definición de reemplazos: Se define un diccionario llamado `reemplazos` que contiene diferentes caracteres que se desean reemplazar en los datos. Esto puede ser útil para eliminar caracteres no deseados o espacios que podrían afectar el análisis posterior.

2. Apertura del archivo de reporte: Se crea un nuevo archivo llamado "Load_report.txt" utilizando la función `open()` con el modo de escritura ("w") y especificando la codificación "utf-8". Este archivo se utilizará para almacenar el reporte de calidad generado.

3. Generación del reporte: Para cada columna en el DataFrame, se realiza lo siguiente:
   - Se extrae la columna `x` y el nombre de la columna `name`.
   - Se escribe en el archivo "Load_report.txt" información relevante, como el nombre de la columna, la cantidad de filas en el DataFrame (`Length`), la cantidad de valores no nulos, la cantidad de valores únicos y la cantidad de valores nulos.
   - Si la columna contiene datos numéricos, se cuenta la cantidad de valores que no son reales (no numéricos) y se agrega esta información al reporte.
   - Si la columna contiene datos de tipo string (texto), se identifican los valores que no son alfanuméricos y se agrega esta información al reporte.

4. Cierre del archivo de reporte: Se cierra el archivo "Load_report.txt" una vez que se ha generado el reporte completo.

5. Corrección de valores anómalos: A continuación, se realiza una corrección específica para el valor anómalo "﻿Araba" encontrado en la columna "Provincia_local". Se busca la fila que contiene ese valor y se reemplaza con el valor correcto "Araba".

6. Rechequeo de existencia de valores anómalos: Finalmente, se vuelve a buscar si hay filas con el valor anómalo "﻿Araba" en la columna "Provincia_local" para verificar que la corrección se haya realizado correctamente.

Es importante realizar este control de calidad para asegurarnos de que los datos estén limpios, coherentes y listos para su análisis. La corrección de valores anómalos y la generación del reporte son parte fundamental del proceso de preparación de los datos para su uso en etapas posteriores del proyecto.

El archivo "Load_report.txt" contiene el reporte de calidad generado y puede ser revisado para identificar cualquier problema o discrepancia en los datos antes de proceder con el análisis y la carga en sistemas de visualización como Power BI u otras herramientas similares.


# Web scrapping

## Scrapping de URL facilitado

In [ ]:
from urllib.request import Request, urlopen

In [ ]:
webpage = 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33791?tip=AM' # URL path to scrap

peticion = Request(webpage, headers={'User-Agent': 'Mozilla/5.0'}) #Establishing connection to the URL
data_page = urlopen(peticion).read().decode('utf8')
print(peticion)
print(type(data_page))

In [ ]:
import json
started = data_page.split('\n]\n}\n,')
st1=[]
for st in started:
    if '"Codigo":"MUN"' in st:
        st1.append(st)
print(f"{len(st1)} registros encontrados")
Data_standard=[]
for item in st1:
    fila={}
    raw = item.split(', "Data":')
    metajson = json.loads(raw[0]+"}")  #Se agrega la llave que no esta incluida en los items intermedios de la lista resutante del split
    metadata=metajson["Nombre"].split(".")
    data={}
    rawData = (raw[1].lstrip("[").rstrip("\n]\n}\n]\n")+"}").split("\n,")
    nAños= len(rawData)
    for i in range (nAños):
        record = json.loads(rawData[i])
        data.update({record["Anyo"]:record["Valor"]})
    fila.update({"ID" : metajson["COD"]})
    fila.update({"Muestra de poblacion: ": metadata[0]})
    fila.update({"Poblado: ": metadata[1]})
    fila.update({"Procedencia de poblacion: ": (" ").join(metadata[2:4])})
    fila.update({"Provincia" : (", ").join(metajson["MetaData"][1]['T3_JerarquiaPadres'])})
    fila.update(data)
    Data_standard.append(fila)
# print(Data_standard,"\n",sep="\n")
pd.DataFrame(Data_standard,index=list(range(len(Data_standard))))

## Web scraping - Scrapping de URL facilitado

En esta etapa del proyecto, estamos realizando web scraping de una URL específica para obtener datos. El objetivo es extraer información de la página web y estructurarla en un formato adecuado para su posterior análisis y utilización.

1. Establecimiento de la conexión y obtención de datos:
   - Se importan las bibliotecas necesarias, incluyendo `urllib.request` para establecer la conexión y `json` para trabajar con los datos en formato JSON.
   - Se proporciona la URL en la variable `webpage` que contiene la página web que deseamos rascar.
   - Se establece una conexión a la URL utilizando la función `Request()` y se especifica un User-Agent para simular una solicitud de navegador, evitando posibles bloqueos por parte del servidor. Luego, se lee el contenido de la página web en formato `utf8`.

2. Separación y filtrado de datos:
   - El contenido de la página web se divide en una lista de elementos utilizando el carácter '\n]\n}\n,' como separador. Se busca la presencia de registros que contengan la clave `"Codigo":"MUN"`, lo que indica que son los datos que nos interesan. Estos registros se recopilan en la lista `st1`.

3. Estructuración de los datos:
   - Para cada elemento en `st1`, se realiza lo siguiente:
     - Se separa el texto en dos partes usando la cadena ', "Data":' como delimitador. La primera parte contiene metadatos sobre la muestra de población, poblado, procedencia de población y provincia, mientras que la segunda parte contiene información de años y sus respectivos valores.
     - Se utiliza `json.loads()` para convertir la primera parte en un diccionario. Se extraen metadatos relevantes y se almacenan en variables.
     - Se divide la segunda parte en una lista de registros JSON correspondientes a cada año y valor.
     - Se itera sobre los registros JSON y se extraen los datos de año y valor, que se almacenan en un diccionario `data`.
     - Se crea un diccionario `fila` para almacenar los metadatos y los datos de año y valor.
     - Se agrega el diccionario `fila` a la lista `Data_standard` que contendrá todos los datos estructurados.

4. Creación de DataFrame:
   - Se crea un DataFrame de pandas a partir de la lista `Data_standard`, que muestra los datos en forma de tabla, listando cada fila con sus respectivos metadatos y valores para cada año.

En esta etapa del proyecto, hemos extraído datos de la página web y los hemos estructurado en un formato tabular (DataFrame). A partir de aquí, podemos realizar análisis, limpieza adicional o cargar los datos en el Data Warehouse para su posterior uso en la presentación y visualización de datos.


## Conexion mediante Api a catalogos

In [ ]:
import requests
keyword="DPOP"
url = f"https://datos.gob.es/apidata/catalog/dataset/keyword/{keyword}"
x = requests.get(url, params= {"_sort": "items,description", "_pageSize": 200})
print("Request response code: ",x.status_code, sep="\n")
respuesta = x.json()

In [ ]:
numero_respuestas = len(respuesta['result']["items"])
accessPoint =[]
for n in range(numero_respuestas):
    description = respuesta['result']["items"][n]["description"][0]["_value"][16:]
    notaccessURL = respuesta['result']["items"][n]["distribution"]
    for i in notaccessURL:
        if "json" in (i["format"]["value"]):
            accessURL = notaccessURL[notaccessURL.index(i)]['accessURL']
    accessPoint.append({"description":description, "URL" :accessURL})
Endpoints = pd.DataFrame(accessPoint, index=list(range(len(accessPoint))))
URLmart = Endpoints[Endpoints["description"].str.contains(": Población por municipios y sexo.")==True]
Endpoints

En esta etapa del proyecto, estamos conectando a través de una API a un catálogo de datos para obtener información relacionada con el término de búsqueda "DPOP". El objetivo es obtener información relevante y estructurarla en un DataFrame para su posterior análisis y utilización.

1. Realización de la solicitud y obtención de datos:
   - Se importa la biblioteca `requests` para realizar la conexión y obtener los datos a través de una solicitud HTTP.
   - Se define la variable `keyword` con el término de búsqueda "DPOP".
   - Se forma la URL de la API a través de la concatenación de la variable `keyword` en la URL base.
   - Se realiza una solicitud GET a la URL de la API y se aplican parámetros adicionales como `_sort` y `_pageSize` para ordenar y limitar la cantidad de resultados devueltos.
   - Se imprime el código de respuesta de la solicitud HTTP para verificar que la conexión fue exitosa.

2. Procesamiento de la respuesta de la API:
   - Se convierte la respuesta de la API en formato JSON utilizando `x.json()` y se almacena en la variable `respuesta`.
   - Se obtiene el número de respuestas (registros) en el catálogo utilizando `len(respuesta['result']["items"])`.
   - Se crea una lista llamada `accessPoint` para almacenar la información relevante que se extraerá del catálogo.

3. Extracción de información relevante:
   - Se recorre cada registro en la respuesta de la API utilizando un bucle `for`.
   - Se obtiene la descripción del registro y se almacena en la variable `description`.
   - Se busca la URL de acceso a los datos en formato JSON dentro del registro y se almacena en la variable `accessURL`.
   - La información de `description` y `accessURL` se agrega al diccionario `accessPoint` en cada iteración del bucle.

4. Creación del DataFrame:
   - Se crea un DataFrame de pandas llamado `Endpoints` a partir de la lista `accessPoint`, que mostrará la descripción y la URL de acceso a los datos para cada registro.
   - Se filtra el DataFrame para obtener solo los registros que contienen la descripción ": Población por municipios y sexo." y se almacena en el DataFrame `URLmart`.

En esta etapa del proyecto, hemos obtenido información específica del catálogo a través de la API y la hemos estructurado en un formato tabular (DataFrame). A partir de aquí, podemos utilizar esta información para realizar análisis, presentar datos o cualquier otra tarea requerida.


## Consulta en serie automatizada

In [ ]:
def requesting(webpage):
    from urllib.request import Request, urlopen
    try:
        peticion = Request(webpage, headers={'User-Agent': 'Mozilla/5.0'}) #Establishing connection to the URL
        data_page = urlopen(peticion).read().decode('utf8')
        return data_page
    except Exception as e:
        print(e)

In [ ]:
def dataframing(data_page, name = "File_result"):
    import json
    started = data_page.split('\n]\n}\n,')
    st1=[]
    for st in started:
        if '"Codigo":"MUN"' in st:
            st1.append(st)
    print(f"{len(st1)} registros encontrados")
    Data_standard=[]
    for item in st1:
        fila={}
        raw = item.split(', "Data":')
        metajson = json.loads(raw[0]+"}")  #Se agrega la llave que no esta incluida en los items intermedios de la lista resutante del split
        metadata=metajson["Nombre"].split(".")
        data={}
        rawData = (raw[1].lstrip("[").rstrip("\n]\n}\n]\n")+"}").split("\n,")
        nAños= len(rawData)
        for i in range (nAños):
            record = json.loads(rawData[i])
            data.update({record["Anyo"]:record["Valor"]})
        fila.update({"ID" : metajson["COD"]})
        fila.update({"Muestra de poblacion: ": metadata[0]})
        fila.update({"Poblado: ": metadata[1]})
        fila.update({"Procedencia de poblacion: ": (" ").join(metadata[2:4])})
        for c in range(len(metajson["MetaData"])):
            if 'T3_JerarquiaPadres' in metajson["MetaData"][c].keys():
                fila.update({"Provincia" : (", ").join(metajson["MetaData"][c]['T3_JerarquiaPadres'])})
        fila.update(data)
        Data_standard.append(fila)
    df = pd.DataFrame(Data_standard,index=list(range(len(Data_standard))))
    df.to_csv('name.csv',sep=",")

In [ ]:
links = URLmart["URL"].values
description = URLmart["description"].values
webpage = links[49]
data_page = requesting(webpage)
final = dataframing(data_page)

En esta etapa del proyecto, se ha creado una función para automatizar el proceso de consulta a través de una lista de enlaces web (URLs) y la creación de DataFrames a partir de los datos obtenidos. El proceso se realiza en dos funciones distintas: `requesting()` y `dataframing()`.

1. Función `requesting(webpage)`:
   - Esta función se encarga de realizar una solicitud HTTP a una URL específica.
   - Utiliza la biblioteca `urllib.request` para establecer la conexión a la URL y obtener los datos de la página web.
   - Si la conexión es exitosa, devuelve los datos de la página web en formato de texto.

2. Función `dataframing(data_page, name="File_result")`:
   - Esta función se encarga de procesar los datos de la página web y estructurarlos en un DataFrame.
   - Los datos de la página web se dividen en una lista llamada `st1`, que contiene registros específicos que contienen la clave `"Codigo":"MUN"`.
   - Se recorre cada registro en `st1`, y se extrae la información relevante como metadatos y valores para cada año. Luego, se estructura en un diccionario llamado `fila`.
   - Se agrega cada diccionario `fila` a la lista `Data_standard`.
   - Finalmente, se crea un DataFrame de pandas llamado `df` a partir de la lista `Data_standard`, y se guarda en un archivo CSV con el nombre especificado en la variable `name`.

3. Automatización de la consulta y creación de DataFrames:
   - Se obtiene la lista de URLs de la variable `links`.
   - Se recorre la lista de URLs en un bucle `for`.
   - En cada iteración, se realiza una consulta a la URL utilizando la función `requesting(webpage)` y se almacenan los datos de la página web en la variable `data_page`.
   - Luego, se utiliza la función `dataframing(data_page)` para procesar los datos y crear el DataFrame correspondiente.

En esta etapa del proyecto, se están automatizando las consultas y creaciones de DataFrames para cada URL en la lista `links`. Cada DataFrame se guarda en un archivo CSV con un nombre específico. Este enfoque es útil cuando se tiene una lista de enlaces web de donde se desea extraer datos de manera automatizada y estructurada para su posterior análisis y manipulación.

Es importante tener en cuenta que el código se encuentra en proceso y puede requerir ajustes o mejoras adicionales según las necesidades específicas del proyecto.


## Obtencion de base de datos para presentacion en PowerBI

In [ ]:
import requests
keyword="Mercado laboral y salarios"
url = f"https://datos.gob.es/apidata/catalog/dataset/keyword/{keyword}"
x = requests.get(url, params= {"_sort": "items,description", "_pageSize": 200, "_page":8})
print("Request response code: ",x.status_code, sep="\n")
respuesta = x.json()
numero_respuestas = len(respuesta['result']["items"])


In [ ]:
accessPoint =[]
for n in range(numero_respuestas):
    description = respuesta['result']["items"][n]["description"][0]["_value"]
    notaccessURL = respuesta['result']["items"][n]["distribution"]
    for i in notaccessURL:
        if "json" in (i["format"]["value"]):
            accessURL = notaccessURL[notaccessURL.index(i)]['accessURL']
    accessPoint.append({"description":description, "URL" :accessURL})
Endpoints = pd.DataFrame(accessPoint, index=list(range(len(accessPoint))))
URLmart = Endpoints[Endpoints["description"].str.contains("Tasas de paro por nivel de formación alcanzado, sexo y grupo de edad. Anual")==True]
URLmart["URL"].values

In [ ]:
for a in range(len(data)):
            print({"Periodo":data[a][0]})
            print({"Total (%)":data[a][1]})

In [ ]:
def dfing2 (data_page):
    import json
    started = data_page.split('\n}\n,')
    Data_standard=[]
    for item in started:
        raw = item.split(', "Data":')
        metajson = json.loads(raw[0].strip().lstrip("[")+"}")
        metadata=metajson["Nombre"].split(".")
        data=[]
        rawData = (raw[1].lstrip("[").rstrip("\n]\n}\n]\n")+"}").split("\n,")
        nAños= len(rawData)
        for i in range (nAños):
            record = json.loads(rawData[i])
            data.append([record["Anyo"],record["Valor"]])
        for a in range(len(data)):
            fila={}
            fila.update({"Sexo" : metadata[2]})
            fila.update({"Edad": metadata[3].strip()})
            fila.update({"Nivel de formacion": metadata[4]})
            fila.update({"Periodo":data[a][0]})
            fila.update({"Total (%)":data[a][1]})
            Data_standard.append(fila)
    return Data_standard

In [ ]:
links = URLmart["URL"].values
for link in links:
    webpage = link
    data_page = requesting(webpage)
    dataset=dfing2(data_page)

Sexo            Edad   Nivel de formación alcanzado  Periodo   Total
Ambos sexos     Total              Total              2022      12,92

In [ ]:
import pandas as pd
df = pd.DataFrame(dataset)
df

En esta etapa del proyecto, se ha realizado la obtención de una base de datos específica relacionada con el "Mercado laboral y salarios" desde una API. El objetivo es estructurar esta información en un DataFrame para su posterior presentación en Power BI o cualquier otra herramienta de visualización.

1. Realización de la solicitud y obtención de datos:
   - Se define la variable `keyword` con el término de búsqueda "Mercado laboral y salarios".
   - Se forma la URL de la API a través de la concatenación de la variable `keyword` en la URL base.
   - Se realiza una solicitud GET a la URL de la API y se aplican parámetros adicionales como `_sort`, `_pageSize` y `_page` para ordenar, limitar la cantidad de resultados y obtener una página específica de resultados.
   - Se imprime el código de respuesta de la solicitud HTTP para verificar que la conexión fue exitosa.

2. Procesamiento de la respuesta de la API:
   - La respuesta de la API se convierte en formato JSON utilizando `x.json()` y se almacena en la variable `respuesta`.
   - Se obtiene el número de respuestas (registros) en el catálogo utilizando `len(respuesta['result']["items"])`.
   - Se crea una lista llamada `accessPoint` para almacenar la información relevante que se extraerá del catálogo.

3. Extracción de información relevante:
   - Se recorre cada registro en la respuesta de la API utilizando un bucle `for`.
   - Se obtiene la descripción del registro y se almacena en la variable `description`.
   - Se busca la URL de acceso a los datos en formato JSON dentro del registro y se almacena en la variable `accessURL`.
   - La información de `description` y `accessURL` se agrega al diccionario `accessPoint` en cada iteración del bucle.

4. Estructuración de la base de datos:
   - Se obtienen los enlaces de acceso a los datos en formato JSON desde el DataFrame `URLmart["URL"].values`.
   - Para cada enlace, se realiza una solicitud a la URL utilizando la función `requesting(webpage)` y se almacenan los datos de la página web en la variable `data_page`.
   - Luego, se utiliza la función `dfing2(data_page)` para procesar los datos y crear una lista de diccionarios llamada `dataset`, que contiene la información estructurada.

5. Creación del DataFrame final:
   - Se crea un DataFrame de pandas llamado `df` a partir de la lista de diccionarios `dataset`.
   - El DataFrame resultante contendrá la información sobre el sexo, edad, nivel de formación alcanzado, período y el total (%) correspondiente.

En esta etapa del proyecto, hemos obtenido una base de datos específica desde la API, la hemos estructurado en un formato tabular (DataFrame) y estamos listos para presentarla y analizarla en Power BI o cualquier otra herramienta de visualización.


# DataMart 

## CSV de provincias standarizado y limpiado

In [ ]:
provinces.to_csv('Provincias.csv')

En esta etapa del proyecto, se ha creado un DataFrame llamado `provinces` que contiene datos sobre provincias obtenidos anteriormente a través del proceso de web scraping y limpieza.

1. Creación del archivo CSV:
   - Se utiliza el método `to_csv()` del DataFrame `provinces` para exportar los datos a un archivo CSV.
   - El nombre del archivo se establece como "Provincias.csv".

2. Guardado del archivo CSV:
   - El archivo CSV se guarda en el directorio de trabajo actual con el nombre "Provincias.csv".
   - Los datos en el archivo CSV estarán organizados y limpios, listos para ser utilizados en otras etapas del proyecto o para su análisis y presentación en herramientas como Power BI.

En esta etapa del proyecto, se ha completado la creación y guardado del archivo CSV que contiene información sobre provincias estandarizada y limpiada. Este archivo CSV puede ser útil para cargar los datos en el Data Warehouse o para su uso directo en herramientas de visualización o análisis de datos como Power BI.
